In the machine learning domain, feature construction is always regarded as a critical component of a machine learning pipeline. Just as the deep neural network improves the performance of the traditional neural network through constructed features, if we can construct some powerful features, we may be able to significantly improve the learning capability of existing simple machine learning algorithms. In fact, in most ML competitions, feature engineering is always regarded as a critical point in winning the competition. Such features, however, necessitate a significant amount of effort on the part of domain experts. As a result, automated feature construction has become a hot topic in recent years. In general, we can generate several features using different methods, such as evolutionary algorithms or random search, and then select a few of them to use on the new machine learning system. In such a process, it is obvious that determining whether a feature is good or bad is a critical issue. As a result of this problem, most feature construction methods can be classified into three types: wrapper-based feature construction, filter-based feature construction, and embedded feature construction.
The difference between these three methods is depicted in the figure below. The first is the wrapper-based feature construction method. It is obvious that this method is based on a specific machine learning algorithm. To be more specific, for a set of features, we must build a machine learning model based on those features and then use the validation score as the score of those features. The second method, the filter-based construction method, does not rely on any machine learning algorithm. In contrast, it estimates the efficacy of each feature using a statistical measure, such as the Pearson correlation between that feature and the response variable. Finally, the third method, embedded feature construction, is only applicable to some machine learning algorithms, such as decision tree based methods, which can provide feature importance. As a result, we can use the feature's important value to select critical features.

These three methods each have their own set of benefits and drawbacks. The wrapper-based feature construction method, for example, is the most reliable method because it directly optimizes the validation score on a specific algorithm. The disadvantage is that it is extremely slow, especially when we need to construct some features for some expensive machine learning algorithms, such as neural networks or gradient boosted decision trees. In contrast, the filter-based construction method is quick because we don't need to train any machine learning algorithms during the scoring process. However, because it does not rely on any machine learning algorithms, the obtained result may not be applicable to the target algorithm. In terms of the last one, the embedded features construction method can, in fact, collaborate with the wrapper-based feature construction method. As a result, we will concentrate on comparing the first two methods in this article.

In [1]:
import random

import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

random.seed(0)
np.random.seed(0)

X, y = load_diabetes(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In this article, I want to clear up a common misconception: unlike most people, I do not believe that the wrapper-based feature construction method is significantly slower than the filter-based feature construction method. In reality, the difference is not that great, and it should be within an order of magnitude. To demonstrate such an assumption, we run a scenario that evaluates a set of five features using Spearman correlation and a decision tree. As shown below, I repeated the experiment 1000 times to ensure the reliability of the experimental results. We can see that the Spearman correlation method must take two seconds to complete this process.

In [2]:
from scipy.stats import spearmanr
import time

st = time.time()
for i in range(1000):
    for j in range(5):
        pseudo_feature = np.random.rand(len(y_train))
        spearmanr(pseudo_feature, y_train)
print('time', time.time() - st)

In comparison, if we build a decision tree based on the construction features and use the cross validation loss as the score of those features, the surprising thing is that we don't have to spend a lot of time on it. In reality, we only need to spend three more times to complete this task. The decision tree algorithm is, in fact, a lightning-fast algorithm. In theory, the decision tree's depth should be O(logN), where N represents the amount of data. Given such a fast algorithm, it's debatable whether we really need a less expensive but unreliable feature score estimation method.

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor

st = time.time()
for i in range(1000):
    pseudo_feature = np.random.rand(len(y_train), 5)
    tree = DecisionTreeRegressor()
    cross_val_score(tree, pseudo_feature, y_train, cv=5)
print('time', time.time() - st)

To investigate the problem further, namely whether it is worthwhile to use three more times to find the useful features, we chose two frameworks in the genetic programming-based feature construction domain to validate the effectiveness of these two paradigms. The evolutionary forest is the first. It is a new framework based on a wrapper-based feature construction paradigm. This algorithm, like the illustrative example, evaluates features using a decision tree. As a result, the obtained features can be applied directly to tree-based algorithms. We found that it will take 27 seconds to obtain the results based on the experimental results.

In [4]:
from evolutionary_forest.forest import EvolutionaryForestRegressor

st = time.time()
r = EvolutionaryForestRegressor(max_height=8, normalize=True, select='AutomaticLexicase',
                                mutation_scheme='weight-plus-cross-global',
                                gene_num=10, boost_size=100, n_gen=20, base_learner='DT',
                                verbose=True)
r.fit(x_train, y_train)
print('time', time.time() - st)

The second is gplearn, which is a well-known genetic programming package. Unlike the previous package, this one employs the Spearman correlation coefficient to determine whether or not a feature is promising. As a result, this package has the advantage of being faster than the wrapper-based feature construction method. However, it is difficult to say whether the built feature is applicable to the target machine learning algorithm. Based on the experimental results, we found that it will take 8 seconds to obtain the results, demonstrating that this paradigm is significantly faster than the wrapper-based one.

In [5]:
from gplearn.genetic import SymbolicTransformer

st = time.time()
s = SymbolicTransformer(population_size=500, generations=20, hall_of_fame=500, n_components=50, metric='spearman',
                        verbose=1)
s.fit(x_train, y_train)
print('time', time.time() - st)

However, we should not focus solely on training time. In fact, whether its discovered features are useful or not is a more pressing concern. As a result, we build three decision models based on original features and those created by the evolutionary forest and gplearn, respectively.
According to the experimental results, the feature obtained by the wrapper-based method can significantly improve the performance of the random forest model. However, the features obtained by the filter-based method do not appear to be as significant.

In [6]:
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.metrics import r2_score
from evolutionary_forest.utils import get_feature_importance, feature_append


def score_func(score_name, model_name, x_train, x_test):
    scores = []
    for i in range(10):
        new_r = {
            'RF': RandomForestRegressor(),
            'AdaBoost': AdaBoostRegressor(),
            'GBDT': GradientBoostingRegressor(subsample=0.8, n_estimators=100),
            'DART': LGBMRegressor(boosting_type='DART', random_state=random.randint(0, 1000), n_estimators=100),
        }[model_name]
        new_r.fit(x_train, y_train)
        score = r2_score(y_test, new_r.predict(x_test))
        all_score.append((score_name, model_name, score))
        scores.append(score)
    return scores

In [7]:
code_importance_dict = get_feature_importance(r, simple_version=False)
new_train_wrapper = feature_append(r, x_train, list(code_importance_dict.keys())[:50], only_new_features=True)
new_test_wrapper = feature_append(r, x_test, list(code_importance_dict.keys())[:50], only_new_features=True)
all_score = []

wrapper_score = score_func('Wrapper', 'RF', new_train_wrapper, new_test_wrapper)
print('r2_score', wrapper_score, np.mean(wrapper_score))

In [8]:
base_score = score_func('Base', 'RF', x_train, x_test)
print('r2_score', base_score, np.mean(base_score))

In [9]:
new_train = s.transform(x_train)
new_test = s.transform(x_test)
filter_score = score_func('Filter', 'RF', new_train, new_test)
print('r2_score', filter_score, np.mean(filter_score))

To further solidify our conclusion, we must conduct a statistical test on the results. In this article, we will use the rank sum test to validate our assumption.

In [10]:
from scipy import stats

print('base_score vs wrapper_score', stats.ranksums(base_score, wrapper_score).pvalue)
print('base_score vs filter_score', stats.ranksums(base_score, filter_score).pvalue)
print('wrapper_score vs filter_score', stats.ranksums(wrapper_score, filter_score).pvalue)

Based on the statistical test, we can see that the best method, wrapper-based feature construction, outperforms its counterpart on this task significantly. Even though it costs twice as much, it is worthwhile to spend the extra time on this task.

In addition, we run these generated features through a variety of machine learning algorithms, including AdaBoost, GBDT, and DART. Based on the final results, we conclude that the features generated by the wrapped-based method are more useful, which can effectively improve the model prediction performance. In contrast, while the filter-based method can improve model performance, the effect is minor.

In [11]:
import seaborn as sns

for model in ['AdaBoost', 'GBDT', 'DART']:
    wrapper_score = score_func('Wrapper', model, new_train_wrapper, new_test_wrapper)
    base_score = score_func('Base', model, x_train, x_test)
    filter_score = score_func('Filter', model, new_train, new_test)
    print(wrapper_score, base_score, filter_score)
data = pd.DataFrame(all_score, columns=[
    'Name',
    'Model',
    'Score',
])
sns.boxplot(x='Name', y='Score', hue='Model', data=data)

Finally, in this article, we look at two popular paradigms in feature construction: the wrapper-based feature construction method and the filter-based feature construction method. The former is more accurate, while the latter is faster. In fact, the purpose of this article is not to argue that we should abandon the filter-based feature construction method; each method has its own set of application scenarios. It is obvious that the filter-based method is better suited to an environment with limited computational resources because it takes less time. The main point of this article is that many people believe that the wrapper-based method is slow. However, we show in this article that such a method is perfectly acceptable in most cases. In the future, we should not consider the feature construction method to be an A or B problem, in my opinion. Perhaps weighing the benefits of both and identifying the most applicable scenarios is the best solution for the development of machine learning techniques.